# Python Refresher Lab — FAANG-Level Hands-On

**Goal:** Refresh Python fundamentals + interview patterns the way FAANG expects ML engineers to code.

**Outcome:** You can write clean, efficient Python, explain complexity, and avoid classic interview gotchas.

---

## How to Start
1. Open this notebook in Colab
2. Complete all **TODO** sections
3. Restart runtime → Run all

---

## Lab Rules (FAANG Style)
- ✅ Complexity for each core function
- ✅ Handle edge cases
- ✅ Prefer O(n) over O(n²) when feasible
- ✅ Add tests
- ❌ No hardcoding outputs


In [56]:
from __future__ import annotations
from collections import Counter, defaultdict, deque
import heapq

# NOTE: Use this helper for quick sanity checks
def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

## Section 1 — Python Idioms & Core Data Structures

### Task 1.1: Comprehensions (no unnecessary loops)

Create (without loops):
- `even_squares`: squares of even numbers from 1..20
- `pairs`: list of (x, x*x) for x=1..10
- `flattened`: flatten `nested`

**Explain:** Why can comprehensions be faster than manual loops?

In [57]:
# TODO:
# Create even_squares
# HINT: range(1, 21), filter even with x % 2 == 0
even_squares = [x**2 for x in range(1, 21) if x % 2 == 0]

# TODO:
# Create pairs
pairs = [(x, x**2) for x in range(1, 11)]
# TODO:
# Flatten nested
nested = [[1, 2], [3, 4], [5, 6]]
# HINT: nested loops inside a single comprehension
flattened = [item for sublist in nested for item in sublist]

check('even_squares', even_squares == [4, 16, 36, 64, 100, 144, 196, 256, 324, 400])
check('pairs', pairs[:3] == [(1, 1), (2, 4), (3, 9)] and pairs[-1] == (10, 100))
check('flattened', flattened == [1, 2, 3, 4, 5, 6])

OK: even_squares
OK: pairs
OK: flattened


### Task 1.2: Counter (frequency counting)

Given `s`, compute:
- character frequency using `Counter`
- top 2 most common characters

**Interview angle:** When is Counter preferable to sorting?

In [58]:
s = 'mississippi'

# TODO:
freq = Counter(s)
top2 = freq.most_common(2)

print(freq)
print(top2)
check('top2', top2[0][0] == 'i' and top2[0][1] == 4)

Counter({'i': 4, 's': 4, 'p': 2, 'm': 1})
[('i', 4), ('s', 4)]
OK: top2


### Task 1.3: defaultdict (grouping)

Group words by first letter.

**HINT:** `defaultdict(list)`

**Gotcha:** Don’t use a normal dict unless you handle missing keys.

In [59]:
words = ['apple', 'apricot', 'banana', 'blueberry', 'cherry', 'avocado']

# TODO:
groups = defaultdict(list)
for word in words:
    groups[word[0]].append(word)    

print(dict(groups))
check('groups[a]', dict(groups)['a'] == ['apple', 'apricot', 'avocado'])

{'a': ['apple', 'apricot', 'avocado'], 'b': ['banana', 'blueberry'], 'c': ['cherry']}
OK: groups[a]


## Section 2 — FAANG Core Patterns

### Task 2.1: Two Sum (Hash Map Pattern)

**FAANG Classic.** Return indices of two numbers that sum to target.

# HINT:
- Use a dict: value -> index
- For each `x`, check if `target-x` has been seen

**Explain:** Why is this O(n) time?

In [60]:
def two_sum(nums, target):
    num_map = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in num_map:
            return [num_map[complement], i]
        num_map[num] = i
    return
check('two_sum_1', two_sum([2, 7, 11, 15], 9) == [0, 1])
check('two_sum_2', two_sum([3, 2, 4], 6) == [1, 2])
check('two_sum_dup', two_sum([3, 3], 6) == [0, 1])

OK: two_sum_1
OK: two_sum_2
OK: two_sum_dup


### Task 2.2: Longest Substring Without Repeating Characters (Sliding Window)

**FAANG Classic/Gotcha:** naive O(n²) solutions time out.

# HINT:
- Keep a map char -> most recent index
- Maintain a window `[l..r]` with unique chars

**Explain:** What invariant does the window maintain?

In [61]:
def length_of_longest_substring(s: str) -> int:
    char_index_map = {}
    left = 0
    max_length = 0  
    for right, char in enumerate(s):
        if char in char_index_map and char_index_map[char] >= left:
            left = char_index_map[char] + 1
        char_index_map[char] = right
        max_length = max(max_length, right - left + 1)
    return max_length

check('ls_abcabcbb', length_of_longest_substring('abcabcbb') == 3)
check('ls_bbbbb', length_of_longest_substring('bbbbb') == 1)
check('ls_pwwkew', length_of_longest_substring('pwwkew') == 3)
check('ls_empty', length_of_longest_substring('') == 0)

OK: ls_abcabcbb
OK: ls_bbbbb
OK: ls_pwwkew
OK: ls_empty


### Task 2.3: Product of Array Except Self (Prefix/Suffix)

**FAANG Classic/Gotcha:**
- No division
- O(n) time

# HINT:
- Build prefix products in one pass
- Multiply by suffix products in reverse pass

**Explain:** Why does this work with zeros?

In [62]:
def product_except_self(nums):
    length = len(nums)
    answer = [0]*length

    # Calculate lefts
    left = 1
    for i in range(length):
        answer[i] = left
        left *= nums[i]

    # Calculate rights
    right = 1
    for i in range(length-1, -1, -1):
        answer[i] *= right
        right *= nums[i]

    return answer 
    

check('pes_1234', product_except_self([1, 2, 3, 4]) == [24, 12, 8, 6])
check('pes_zero', product_except_self([-1, 1, 0, -3, 3]) == [0, 0, 9, 0, 0])

OK: pes_1234
OK: pes_zero


## Section 3 — FAANG Hard (Optional / Bonus)

### Task 3.1: Sliding Window Maximum (Deque)

**FAANG Classic:** maintain a decreasing deque of indices.

# HINT:
- Pop from left if out of window
- Pop from right while current value is larger

**Explain:** Why is this amortized O(n)?

In [63]:
def max_sliding_window(nums, k):
    # Deque to store indices
    deq = deque()
    result = []
    for i in range(len(nums)):
        # Remove indices out of the current window
        if deq and deq[0] < i - k + 1:
            deq.popleft()
        
        # Remove smaller values at the end as they are not useful
        while deq and nums[deq[-1]] < nums[i]:
            deq.pop()
        
        deq.append(i)
        
        # Append the current max to the result list
        if i >= k - 1:
            result.append(nums[deq[0]])
    return result

check('msw', max_sliding_window([1,3,-1,-3,5,3,6,7], 3) == [3,3,5,5,6,7])

OK: msw


### Task 3.2: LRU Cache (Hash Map + Doubly Linked List)

**FAANG Hard/Gotcha:** You must get O(1) `get` and `put`.

# HINT:
- Dict: key -> node
- Doubly linked list: move-to-front on access

You can implement this later if you run out of time today.

In [64]:
class Node:
    def __init__(self, key: int, value: int):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None


class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}
        self.head = None
        self.tail = None

    def remove_node(self, node: Node) -> None:
        prev_node = node.prev
        next_node = node.next

        if prev_node:
            prev_node.next = next_node
        else:
            self.head = next_node

        if next_node:
            next_node.prev = prev_node
        else:
            self.tail = prev_node

        node.prev = None
        node.next = None

    def add_to_head(self, node: Node) -> None:
        node.prev = None
        node.next = self.head

        if self.head:
            self.head.prev = node
        else:
            self.tail = node

        self.head = node

    def get(self, key: int) -> int:
        if key in self.cache:
            node = self.cache[key]
            self.remove_node(node)
            self.add_to_head(node)
            return node.value
        return -1

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            node = self.cache[key]
            node.value = value
            self.remove_node(node)
            self.add_to_head(node)
        else:
            new_node = Node(key, value)
            if len(self.cache) >= self.capacity:
                lru_key = self.tail.key
                self.remove_node(self.tail)
                del self.cache[lru_key]
            self.add_to_head(new_node)
            self.cache[key] = new_node


lru = LRUCache(2)
lru.put(1, 1)
lru.put(2, 2)
check('lru_get_1', lru.get(1) == 1)
lru.put(3, 3)    # evicts key 2
check('lru_get_2', lru.get(2) == -1)
lru.put(4, 4)    # evicts key 1
check('lru_get_1_evicted', lru.get(1) == -1)
check('lru_get_3', lru.get(3) == 3)
check('lru_get_4', lru.get(4) == 4) 

OK: lru_get_1
OK: lru_get_2
OK: lru_get_1_evicted
OK: lru_get_3
OK: lru_get_4


---

## Submission Checklist
- All TODOs completed
- All tests passing
- Explain prompts answered
- Complexity noted
